In [284]:
# Needed modules :
import os
import gc
import copy
import sklearn

import Drive

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt


from sys import getsizeof
from typing import List

import Preprocessing as pre
from functools import reduce

In [265]:
# Function definitions :

def time_indexed_df(df1: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    _tmp = copy.deepcopy(df1)
    _tmp.index = df1.dateTime
    _tmp.drop('dateTime', axis=1, inplace=True)
    _tmp = _tmp.sort_index()
    gc.collect()
    return _tmp

def fill_nas(df1: pd.core.frame.DataFrame, 
             col_names: List[str] = ['activeInsulin', 'carbs', 'insulin', 'trend']) -> pd.core.frame.DataFrame:
    _tmp = copy.deepcopy(df1)
    for name in col_names:
        _tmp[name] = df1[name].fillna(0)
    gc.collect()
    return _tmp


In [266]:
d = Drive.Drive()

In [267]:
file_name = 'journal.jl'
file_path = os.path.join('data', file_name)
d.download(file_name   = file_name, 
           target_name = file_path
          )

True

In [268]:
pre.file_filter(file_path)

In [269]:
ls data/

journal_filtered.jl  journal.jl


In [270]:
_raw = pd.read_json(file_path, lines=True)
_raw.head()

,BG,IG,LOT,REF,activeInsulin,carbs,dateTime,details,food,initSuccess,insulin,secondRound,trend,type
0,227.0,226.0,NaN,NaN,NaN,0.0,2019-02-07 00:27:00,Normal,None,NaN,1.05,NaN,0.0,data
1,NaN,190.0,NaN,NaN,NaN,NaN,2019-02-07 01:26:00,Normal,None,NaN,NaN,NaN,NaN,data
2,NaN,174.0,NaN,NaN,NaN,NaN,2019-02-07 08:49:00,Wake,None,NaN,NaN,NaN,0.0,event
3,NaN,102.0,NaN,NaN,NaN,NaN,2019-02-07 05:04:00,Suspension before hypo,None,NaN,NaN,NaN,0.0,alarm
4,NaN,98.0,NaN,NaN,NaN,NaN,2019-02-07 06:25:00,Basal resumption,None,NaN,NaN,NaN,0.0,alarm


In [271]:
#_raw[_raw.index.duplicated()]

In [304]:
undesired_columns = [ 
    'LOT',
    'REF', 
    'initSuccess', 
    'secondRound',
    'food'
]

In [305]:
_tmp = (
        _raw.drop(undesired_columns, axis=1)
       ).loc[
                (_raw['type'] == 'data') |
                (_raw['type'] == 'event')
            ]

In [306]:
_t_data = time_indexed_df(_tmp)
_tmp.head()

,BG,IG,activeInsulin,carbs,dateTime,details,insulin,trend,type
0,227.0,226.0,NaN,0.0,2019-02-07 00:27:00,Normal,1.05,0.0,data
1,NaN,190.0,NaN,NaN,2019-02-07 01:26:00,Normal,NaN,NaN,data
2,NaN,174.0,NaN,NaN,2019-02-07 08:49:00,Wake,NaN,0.0,event
5,NaN,148.0,NaN,NaN,2019-02-07 02:24:00,Normal,NaN,NaN,data
6,NaN,131.0,NaN,NaN,2019-02-07 03:14:00,Normal,NaN,NaN,data


In [307]:
_t_data.head()

,BG,IG,activeInsulin,carbs,details,insulin,trend,type
dateTime,,,,,,,,
2019-02-07 00:27:00,227.0,226.0,NaN,0.0,Normal,1.05,0.0,data
2019-02-07 01:26:00,NaN,190.0,NaN,NaN,Normal,NaN,NaN,data
2019-02-07 02:24:00,NaN,148.0,NaN,NaN,Normal,NaN,NaN,data
2019-02-07 03:14:00,NaN,131.0,NaN,NaN,Normal,NaN,NaN,data
2019-02-07 04:24:00,NaN,118.0,NaN,NaN,Normal,NaN,NaN,data


In [308]:
data = fill_nas(_t_data)
data.head()

,BG,IG,activeInsulin,carbs,details,insulin,trend,type
dateTime,,,,,,,,
2019-02-07 00:27:00,227.0,226.0,0.0,0.0,Normal,1.05,0.0,data
2019-02-07 01:26:00,NaN,190.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 02:24:00,NaN,148.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 03:14:00,NaN,131.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 04:24:00,NaN,118.0,0.0,0.0,Normal,0.00,0.0,data


In [332]:
data2 = copy.deepcopy(data)

In [366]:
data2['BG'] = list( 
                    map(
                         lambda x, y: x if not np.isnan(x) else y, data['IG'], data['BG']
                       )
                  )                 

In [359]:
(lambda x, y: x if not np.isnan(x) else y)(4, np.nan)

4

In [362]:
x = map( lambda x, y: x if not np.isnan(x) else y, data['IG'], data['BG'] )

In [365]:
data2['BG']

dateTime
2019-02-07 00:27:00    226.0
2019-02-07 01:26:00    190.0
2019-02-07 02:24:00    148.0
2019-02-07 03:14:00    131.0
2019-02-07 04:24:00    118.0
2019-02-07 08:49:00    174.0
2019-02-07 09:24:00    184.0
2019-02-07 11:24:00    195.0
2019-02-07 12:53:00    133.0
2019-02-07 15:32:00    169.0
2019-02-07 16:33:00    143.0
2019-02-07 17:39:00     99.0
2019-02-07 18:13:00    101.0
2019-02-07 19:38:00    134.0
2019-02-07 20:41:00    140.0
2019-02-07 22:37:00    207.0
2019-02-08 10:12:00    206.0
2019-02-08 12:41:00    120.0
2019-02-08 20:00:00    100.0
2019-02-08 21:31:00    220.0
2019-02-08 23:30:00    152.0
2019-02-09 09:27:00     90.0
2019-02-09 10:52:00    105.0
2019-02-09 12:20:00    169.0
2019-02-09 17:05:00     90.0
2019-02-09 20:35:00    180.0
2019-02-09 22:36:00    149.0
2019-02-10 00:50:00    124.0
2019-02-10 09:00:00     95.0
2019-02-10 09:58:00    182.0
                       ...  
2019-02-19 16:54:00    215.0
2019-02-19 19:34:00    169.0
2019-02-19 20:39:00    115.0
2019-

In [363]:
for i in x:
    print(i)

226.0
190.0
148.0
131.0
118.0
174.0
184.0
195.0
133.0
169.0
143.0
99.0
101.0
134.0
140.0
207.0
206.0
120.0
100.0
220.0
152.0
90.0
105.0
169.0
90.0
180.0
149.0
124.0
95.0
182.0
153.0
130.0
114.0
82.0
98.0
81.0
299.0
299.0
174.0
164.0
127.0
96.0
104.0
121.0
174.0
208.0
218.0
241.0
234.0
137.0
189.0
133.0
122.0
63.0
69.0
98.0
144.0
154.0
127.0
155.0
146.0
209.0
nan
141.0
123.0
150.0
101.0
196.0
187.0
89.0
85.0
124.0
111.0
104.0
175.0
161.0
211.0
200.0
95.0
97.0
174.0
164.0
159.0
137.0
85.0
106.0
264.0
330.0
100.0
126.0
103.0
103.0
83.0
179.0
148.0
129.0
113.0
91.0
95.0
150.0
158.0
194.0
173.0
140.0
128.0
115.0
97.0
176.0
93.0
115.0
132.0
89.0
197.0
120.0
229.0
220.0
220.0
215.0
169.0
115.0
149.0
124.0
121.0
94.0
157.0
208.0
168.0
175.0
72.0
209.0
140.0
107.0
108.0
115.0
103.0
98.0
83.0
81.0
nan
157.0
162.0
183.0
153.0
147.0
199.0
120.0
167.0


In [323]:
x = [ i for i in range(50) ]

In [324]:
y = [ i for i in range(50, 100)]

In [330]:
y = pd.core.series.Series(y)

In [349]:
z = pd.core.series.Series( list(
                            map(
                                lambda x, y: x+y, x, y
                                )
                            )
                     )

In [350]:
type(z)

pandas.core.series.Series

In [351]:
z

0     181.0
1     169.0
2     226.0
3     237.0
4     249.0
5     188.0
6     225.0
7     200.0
8     157.0
9     160.0
10    194.0
11    201.0
12    290.0
13    267.0
14    184.0
15    165.0
16    295.0
17    219.0
18    138.0
19    174.0
20    239.0
21    161.0
22    253.0
23    222.0
24    198.0
25    173.0
26    246.0
27    230.0
28    208.0
29    193.0
30    162.0
31    179.0
32    163.0
33    382.0
34    383.0
35    259.0
36    250.0
37    214.0
38    184.0
39    199.0
40    211.0
41    279.0
42    300.0
43    316.0
44    335.0
45    341.0
46    233.0
47    286.0
48    231.0
49    221.0
dtype: float64